# Librerías

In [ ]:
pip install biopython

In [19]:
from Bio.Seq import transcribe, translate
from Bio.Seq import Seq
from Bio import SeqIO

# Clases

In [20]:
# ruta al archivo FASTA
ruta_fasta = "../data/cas9_adn.fa"

# leer la secuencia del archivo FASTA
with open(ruta_fasta, "r") as archivo:
    #guardar la secuencia de aminoacidos de cas9
    secuencia_cas9 = (next(SeqIO.parse(archivo, "fasta")).seq).translate()

In [21]:
class Cas9:    
    
    def __init__(self, aminoacidos, arn_guia = ""):
        if ( aminoacidos != secuencia_cas9):
            raise ValueError("la cadena de aminoacidos no es una proteina cas9")
        else:
            self.__aminoacidos = aminoacidos
            self.__arn_guia = arn_guia

    @property
    def aminoacidos(self):
        return self.__aminoacidos

    @aminoacidos.setter
    def aminoacidos(self, aminoacidos):
        self.__aminoacidos = aminoacidos

    @property
    def arn_guia(self):
        return self.__arn_guia

    @arn_guia.setter
    def arn_guia(self, arn_guia):
        self.__arn_guia = arn_guia

    def __str__(self):
        return f'Aminoscidos:{self.__aminoacidos} \ARN_Guia: {self.__arn_guia}'

#-------------------------Métodos-----------------------------------
    
    @staticmethod
    def cortar(genoma, arn_guia):
        '''Corta el genoma cuando encuentre una secuencia
            que coincida con el arn_guia .
    
        Parameters
        ----------
        genoma (Seq): secuencia de adn
        arn_gua (Seq): secuencia de arn

        Returns
        ------    
        genoma_cortado (Seq): secuencia adn con un '^' 
            donde se le haya indicado cortar
        '''
        # traducir arn a adn
        adn_guia = arn_guia.transcribe().back_transcribe()
        # buscar en el genoma la secuencia y cortar 
        if adn_guia in genoma:
            corte_index = genoma.find(adn_guia) + len(adn_guia)
            genoma_cortado = genoma[:corte_index] + "^" + genoma[corte_index:]
            return genoma_cortado
        else:
            raise ValueError("la secuencia arn_guia no se encuentra en el genoma")



In [15]:
class Celula:
    
    def __init__(self, genoma, adn_extra = "", proteina_extra = "", ):
        self.__genoma = genoma
        self.__adn_extra = adn_extra
        self.__proteina_extra = proteina_extra
    
    @property
    def genoma(self):
        return self.__genoma

    @genoma.setter
    def genoma(self, genoma):
        self.__genoma = genoma

    @property
    def proteina_extra(self):
        return self.__proteina_extra

    @proteina_extra.setter
    def proteina_extra(self, proteina_extra):
        self.__proteina_extra = proteina_extra.aminoacidos
        #si le introducen una cas9, va a cortar el genoma
        if (self.__proteina_extra == secuencia_cas9):
            self.__genoma = Cas9.cortar(self.__genoma, proteina_extra.arn_guia)
        else: 
            print("la proteina introducida no es una cas9")
     
    @property
    def adn_extra(self):
        return self.__adn_extra

    @adn_extra.setter
    def adn_extra(self, adn_extra):
        self.__adn_extra = adn_extra
        #si el genoma está cortado lo va a reparar con el adn extra que le introdujeron
        if ("^" in self.__genoma):
            posicion_corte = self.__genoma.find("^")
            gen_cortado = self.__genoma[posicion_corte -6 : posicion_corte + 6]
            nueva_secuencia = self.__adn_extra[6:len(self.__adn_extra) -6]
            self.__genoma = self.__genoma.replace("^", nueva_secuencia)

    def __str__(self):
        return f'Genoma:{self.__genoma} \nADN_extra: {self.__adn_extra} \nProteína_extra: {self.__proteina_extra}'


# Main

In [23]:
#crear a cas9
#guardar la secuencia de adn en un formato FASTA descargada
ruta_fasta = "../data/cas9_adn.fa"
with open(ruta_fasta, "r") as archivo:
    adn = next(SeqIO.parse(archivo, "fasta")).seq
print(adn)
print("")
arn = adn.transcribe()
print(arn)
print("")
aminoacidos = arn.translate()
print(aminoacidos)
#crear un objeto de la clase Cas9 con la secuencia de aminoacidos que tengo
cas9 = Cas9(aminoacidos)

ATGGACAAGAAGTATAGCATCGGGCTGGACATTGGAACGAACTCGGTTGGTTGGGCTGTGATTACGGACGAATACAAGGTGCCATCCAAGAAGTTTAAGGTCCTGGGAAACACCGACCGTCACTCAATCAAGAAGAATCTCATTGGAGCCCTGCTCTTCGATAGTGGGGAGACCGCCGAAGCTACTCGACTGAAGCGAACGGCTCGCCGGCGTTATACACGACGCAAGAATCGCATCTGCTACCTCCAGGAGATTTTCAGCAACGAAATGGCTAAGGTTGATGACTCATTCTTTCATCGACTCGAAGAAAGTTTCTTGGTCGAGGAGGATAAGAAGCACGAGCGCCATCCGATCTTTGGTAACATTGTGGATGAGGTTGCCTATCACGAAAAGTACCCAACTATCTATCATCTTCGTAAGAAGCTGGTCGATAGCACGGACAAGGCTGATTTGCGACTTATCTACCTGGCACTCGCGCACATGATTAAGTTCCGCGGCCATTTTCTTATCGAGGGTGACCTGAACCCCGATAATTCTGACGTTGATAAGCTCTTCATCCAGTTGGTCCAAACCTACAATCAGCTGTTTGAGGAAAACCCTATTAATGCATCTGGCGTGGACGCCAAGGCTATCCTTTCGGCGCGCCTGTCTAAGTCGCGGCGTTTGGAGAACCTTATCGCACAACTCCCCGGCGAAAAGAAGAACGGCCTCTTCGGTAATTTGATTGCGTTGTCACTTGGTCTGACTCCTAACTTCAAGAGTAATTTTGACCTGGCAGAGGATGCGAAGCTCCAGTTGTCTAAGGATACGTATGATGACGATCTCGACAACTTGCTTGCCCAAATCGGTGACCAGTACGCTGATCTTTTCCTGGCCGCTAAGAATCTCTCAGATGCAATCCTGCTCAGTGACATTTTGCGGGTCAACACCGAGATTACTAAGGCCCCCCTGTCAGCTAGTATGATCAAGCGGTATGATGAGCACCATCAGGACCTCACCT

In [24]:
#cambiar el genoma de una celula utilizando a cas9
#creo un objeto tipo celula, con una secuencia de adn que sería el genoma
celula = Celula(Seq("GGCATTGCATTTATCATCGGTGTTTGCTGCTTCCTTCTT"))
print(celula.genoma)
#le meto el arn guia a mi objeto Cas9
cas9.arn_guia = Seq("UUUAUCAUC")
#le meto el objeto Cas9 a mi objeto Celula
celula.proteina_extra = cas9
print(celula.genoma)
#le meto la plantilla a mi objeto Celula
#note que se le agrega {} para poder visualizar mejor el cambio
celula.adn_extra = Seq("ATCATC{TTT}GGTGTT")
print(celula.genoma)

GGCATTGCATTTATCATCGGTGTTTGCTGCTTCCTTCTT
GGCATTGCATTTATCATC^GGTGTTTGCTGCTTCCTTCTT
GGCATTGCATTTATCATC{TTT}GGTGTTTGCTGCTTCCTTCTT
